## Visualize word2vec embeddings in tensorboard

In [1]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import numpy as np
import gensim

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

### Load the saved word2vec model

In [2]:
fname = "w2v_model"
model = gensim.models.keyedvectors.KeyedVectors.load(fname)

In [3]:
max_size = len(model.wv.vocab)-1

In [4]:
w2v = np.zeros((max_size,model.layer1_size))

In [5]:
w2v

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [7]:
with open("metadata.tsv", 'w+') as file_metadata:
    for i,word in enumerate(model.wv.index2word[:max_size]):
        w2v[i] = model.wv[word]
        file_metadata.write(word + '\n')

In [8]:
w2v

array([[ 0.38341904,  0.17210738,  0.0855258 , ..., -0.24067372,
         0.08410006,  0.24551795],
       [-0.05418106,  0.28122011,  0.31881899, ..., -0.3409965 ,
         0.00467235,  0.51375943],
       [ 0.74657673,  0.08961128, -0.37936884, ..., -0.07920916,
        -0.4161987 ,  0.34416446],
       ..., 
       [-0.43247166,  0.43675452,  0.13454761, ..., -0.9457109 ,
         0.3885459 , -0.18765165],
       [-0.50884265,  0.69198948, -0.79100227, ..., -0.42207813,
        -0.83687699,  0.10216013],
       [-0.01109237,  0.79544038,  0.41477296, ..., -0.32086468,
         0.18253611,  0.37175721]])

In [9]:
sess = tf.InteractiveSession()

In [10]:
#Let us create a 2D tensor called embedding that holds our embeddings.
with tf.device("/cpu:0"):
    embedding = tf.Variable(w2v, trainable=False, name='embedding')


In [11]:
tf.global_variables_initializer().run()

In [12]:
path = 'tensorboard'

In [13]:
# let us create an object to Saver class which is actually used to 
#save and restore variables to and from our checkpoints
saver = tf.train.Saver()

In [14]:
# using file writer, we can save our summaries and events to our event file.
writer = tf.summary.FileWriter(path, sess.graph)

In [15]:
# adding into projector
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = 'embedding'
embed.metadata_path = 'metadata.tsv'

In [16]:
# Specify the width and height of a single thumbnail.
projector.visualize_embeddings(writer, config)

saver.save(sess, path+'/model.ckpt', global_step=max_size)

'tensorboard/model.ckpt-28070'